# Interpretability With Tensorflow On Azure Machine Learning Service (Local)

## Overview of Tutorial
This notebook is Part 4 (Explaining Your Model Using Interpretability) of a four part workshop that demonstrates an end-to-end workflow for using Tensorflow on Azure Machine Learning Service. The different components of the workshop are as follows:

- Part 1: [Preparing Data and Model Training](https://github.com/microsoft/bert-stack-overflow/blob/master/1-Training/AzureServiceClassifier_Training.ipynb)
- Part 2: [Inferencing and Deploying a Model](https://github.com/microsoft/bert-stack-overflow/blob/master/2-Inferencing/AzureServiceClassifier_Inferencing.ipynb)
- Part 3: [Setting Up a Pipeline Using MLOps](https://github.com/microsoft/bert-stack-overflow/tree/master/3-ML-Ops)
- Part 4: [Explaining Your Model Interpretability](https://github.com/microsoft/bert-stack-overflow/blob/master/4-Interpretibility/IBMEmployeeAttritionClassifier_Interpretability.ipynb)

_**This notebook showcases how to use the Azure Machine Learning Interpretability SDK to train and explain a binary classification model locally.**_

## What is Azure Machine Learning Service?
Azure Machine Learning service is a cloud service that you can use to develop and deploy machine learning models. Using Azure Machine Learning service, you can track your models as you build, train, deploy, and manage them, all at the broad scale that the cloud provides.
![](./images/aml-overview.png)


## What Is Machine Learning Interpretability?
Interpretability is the ability to explain why your model made the predictions it did. The Azure Machine Learning service offers various interpretability features to help accomplish this task. These features include:

- Feature importance values for both raw and engineered features.
- Interpretability on real-world datasets at scale, during training and inference.
- Interactive visualizations to aid you in the discovery of patterns in data and explanations at training time.

By accurately interpretabiliting your model, it allows you to:

- Use the insights for debugging your model.
- Validate model behavior matches their objectives.
- Check for for bias in the model.
- Build trust in your customers and stakeholders.

![](./images/interpretability-architecture.png)

## Change Tensorflow and Interpret Library Versions

We will be using an older version (1.14) for this particular tutorial in the series as Tensorflow 2.0 is not yet supported for Interpretibility on Azure Machine Learning service. We will also be using version 0.1.0.4 of the interpret library. 

If haven't already done so, please update your library versions.

In [ ]:
# %pip uninstall tensorflow-gpu keras --yes
# %pip install tensorflow-gpu==1.14 interpret-community==0.1.0.4
%pip install interpret-community==0.18.0

After installing packages, you must close and reopen the notebook as well as restarting the kernel.

Let's make sure we have the right verisons

In [ ]:
import tensorflow as tf
import interpret_community

print(tf.version.VERSION)

## Train Model
For this tutorial, we will be using the *tf.keras module* to train a basic feed forward neural network on the IBM Employee Attrition Dataset. 

**We will start by writing the training script to train our model**

In [ ]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

def preprocess_data(data):
    '''
    
    '''
    # Dropping Employee count as all values are 1 and hence attrition is independent of this feature
    data = data.drop(['EmployeeCount'], axis=1)
    
    # Dropping Employee Number since it is merely an identifier
    data = data.drop(['EmployeeNumber'], axis=1)
    data = data.drop(['Over18'], axis=1)

    # Since all values are 80
    data = data.drop(['StandardHours'], axis=1)

    # Converting target variables from string to numerical values
    target_map = {'Yes': 1, 'No': 0}
    data["Attrition_numerical"] = data["Attrition"].apply(lambda x: target_map[x])
    target = data["Attrition_numerical"]

    data.drop(['Attrition_numerical', 'Attrition'], axis=1, inplace=True)
    
    # Creating dummy columns for each categorical feature
    categorical = []
    for col, value in data.iteritems():
        if value.dtype == 'object':
            categorical.append(col)

    # Store the numerical columns in a list numerical
    numerical = data.columns.difference(categorical)   

    # We create the preprocessing pipelines for both numeric and categorical data.
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocess = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numerical),
            ('cat', categorical_transformer, categorical)])
    
    pipeline = make_pipeline(preprocess)

    # Split data into train and test sets
    x_train, x_test, y_train, y_test = train_test_split(data, 
                                                        target, 
                                                        test_size=0.2,
                                                        random_state=0,
                                                        stratify=target)
    
    return x_train, x_test, y_train, y_test, pipeline, preprocess
    
# Load and preprocess data
attrition_data = pd.read_csv('./data/data.csv')
x_train, x_test, y_train, y_test, pipeline, preprocess = preprocess_data(attrition_data)

# Transform data
x_train_t = pipeline.fit_transform(x_train)
x_test_t = pipeline.transform(x_test)

# Create model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=16, activation='relu', input_shape=(x_train_t.shape[1],)))
model.add(tf.keras.layers.Dense(units=16, activation='relu'))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy']) 

# Fit model
model.fit(x_train_t, y_train, epochs=20, verbose=1, batch_size=128, validation_data=(x_test_t, y_test))

## Explain Model Locally

We will start by explaining the trained model locally.

**Instantiate the explainer object using trained model.**

In [ ]:
# from interpret.ext.greybox import DeepExplainer

# explainer = DeepExplainer(model,
#                           x_train,
#                           features=x_train.columns,
#                           classes=["STAYING", "LEAVING"], 
#                           transformations = preprocess,
#                           model_task="classification",
#                           is_classifier=True)

import shap

explainer = shap.DeepExplainer(model, x_train)

In [ ]:
# init the JS visualization code
shap.initjs()

# transform the indexes to words
import numpy as np
words = imdb.get_word_index()
num2word = {}
for w in words.keys():
    num2word[words[w]] = w
x_test_words = np.stack([np.array(list(map(lambda x: num2word.get(x, "NONE"), x_test[i]))) for i in range(10)])

# plot the explanation of the first prediction
# Note the model is "multi-output" because it is rank-2 but only has one column
shap.force_plot(explainer.expected_value[0], shap_values[0][0], x_test_words[0])

In [ ]:
masker = shap.maskers.Text(tokenizer, mask_token = "...", collapse_mask_token=True)
explainer = shap.Explainer(model, masker)


**Generate global explanations**

In [ ]:
# Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
# x_train can be passed as well, but with more examples explanations will take longer although they may be more accurate
global_explanation = explainer.explain_global(x_test)

In [ ]:
# Print out a dictionary that holds the sorted feature importance names and values
print('global importance rank: {}'.format(global_explanation.get_feature_importance_dict()))

In [ ]:
# Per class feature names
print('ranked per class feature names: {}'.format(global_explanation.get_ranked_per_class_names()))

# Per class feature importance values
print('ranked per class feature values: {}'.format(global_explanation.get_ranked_per_class_values()))

**Generate local explanations**

In [ ]:
# You can pass a specific data point or a group of data points to the explain_local function
# E.g., Explain the first data point in the test set
instance_num = 1
local_explanation = explainer.explain_local(x_test[:instance_num])

sorted_local_importance_values = local_explanation.get_ranked_local_values()
sorted_local_importance_names = local_explanation.get_ranked_local_names()

print('local importance values: {}'.format(sorted_local_importance_values))
print('local importance names: {}'.format(sorted_local_importance_names))

**Visualize our explanations**

In [ ]:
from interpret_community.widget import ExplanationDashboard
from interpret_community.common.model_wrapper import wrap_model
from interpret_community.dataset.dataset_wrapper import DatasetWrapper
from sklearn.pipeline import Pipeline

wrapped_model, ml_domain = wrap_model(model, DatasetWrapper(x_test_t), "classification")
wrapped_model.fit = model.fit
dashboard_pipeline = Pipeline(steps=[('preprocess', preprocess), ('network', wrapped_model)])
ExplanationDashboard(global_explanation, dashboard_pipeline, datasetX=x_test)